## 株価が右肩上がりで出来高が多く、勢いのある新興銘柄とは？

参照；[資産4億円の投資家　「取引は数分、勢いに乗る」｜マネー研究所｜NIKKEI STYLE](https://style.nikkei.com/article/DGXMZO05078520R20C16A7000000)

> 狙い目は株価が右肩上がりで出来高が多く、勢いのある新興銘柄。最近ではアカツキ（東マ・3932）や、アキュセラ・インク（東マ・4589）などで勝負している。勢いのない銘柄や、右肩下がりの銘柄はGD後、そのまま下がり続けることが多いので相手にしない。

と書いてあるが，** 狙い目は株価が右肩上がりで出来高が多く、勢いのある新興銘柄。** とは何か？



## 仮説①

N日間の終値前日比平均がポジティブであり，N日間の出来高前日比平均がポジティブであること

In [ ]:
# 前日の引け値から当日寄り付きの差

from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing 
from quantopian.pipeline.factors import SimpleMovingAverage, Returns, DailyReturns
from quantopian.pipeline.filters.morningstar import Q1500US,Q500US
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.classifiers.morningstar import Sector 

import numpy as np
import pandas as pd 

class ValueAtNdaysBefore(CustomFactor):
    def compute(self, today, assets, out, values):
        out[:] = values[0]
    

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[-1] * shares[-1])

        
class MAMarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    #window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(np.mean(close * shares, 0))

        
class NumOfForSale(CustomFactor):
    inputs = [Fundamentals.available_for_sale_securities]
    window_length = 1 
    def compute(self, today, assets, out, forsale):
        out[:] = forsale[-1]
        
class FloatingStocks(CustomFactor):
    inputs = [USEquityPricing.volume, Fundamentals.shares_outstanding]
    window_length = 1
    def compute(self, today, assets, out, volume, shares): 
        out[:] = volume[-1] / shares[-1]

class MAPctChange(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean()#.apply(np.log)
        out[:] = mean.values
        
class MAPctChangeLog(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean().apply(np.log)
        out[:] = mean.values
        
class BoolMAPctChangePositive(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean() > 0
        out[:] = mean.values
        

In [ ]:
pipe = Pipeline()

pipe.add(MarketCap(), 'MarketCap')
pipe.add(MAMarketCap(window_length=3), 'MarketCapMA3')
pipe.add(DailyReturns(), 'DailyReturns')
# pipe.add(NumOfForSale(), 'ForSale')
pipe.add(FloatingStocks(), 'FloatingStocks')
pipe.add(Sector(), 'Sector')

pipe.add(ValueAtNdaysBefore(inputs=[USEquityPricing.open], window_length=2), 'Open 2days before')
pipe.add(ValueAtNdaysBefore(inputs=[USEquityPricing.open], window_length=1), 'Open 1day before')
pipe.add(USEquityPricing.open.latest, 'Open')

term = 5

VolumeMAPctChange = MAPctChange(inputs=[USEquityPricing.volume], window_length=term)
CloseMAPctChange = MAPctChange(inputs=[USEquityPricing.close], window_length=term)
VolumeMAPctChangeLog = MAPctChangeLog(inputs=[USEquityPricing.volume], window_length=term)
CloseMAPctChangeLog = MAPctChangeLog(inputs=[USEquityPricing.close], window_length=term)
VolumeMAZscore = SimpleMovingAverage(inputs=[USEquityPricing.volume], window_length=term).zscore()

pipe.add(VolumeMAPctChange, 'VolumeMAPctChange')
pipe.add(CloseMAPctChange, 'CloseMAPctChange')
pipe.add(VolumeMAPctChangeLog, 'VolumeMAPctChangeLog')
pipe.add(CloseMAPctChangeLog, 'CloseMAPctChangeLog')
pipe.add(VolumeMAZscore,'VolumeMAZscore')

BVolumeMAPctChange = BoolMAPctChangePositive(inputs=[USEquityPricing.volume], window_length=term)
BCloseMAPctChange = BoolMAPctChangePositive(inputs=[USEquityPricing.close], window_length=term)
pipe.add(BVolumeMAPctChange, 'BVolumeMAPctChange')
pipe.add(BCloseMAPctChange, 'BCloseMAPctChange')

x = (BVolumeMAPctChange >0) & (BCloseMAPctChange > 0)
pipe.set_screen(Q500US() &  x)

start = "2016-1-1"
end = "2016-6-1"
results = run_pipeline(pipe, start_date= start, end_date=end)

In [ ]:
results.loc["2016-1-5"]

In [ ]:
def merged(results):
    d = dict()
    dates = results.index.get_level_values(0).unique()
    cnt = len(dates)
    
    for i, date in enumerate(dates):
        if i+1 < cnt:
            day1 = date
            day2 = dates[i+1]

            tickers = results.loc[day1].index 
            df = get_pricing(tickers, start_date=day1, end_date=day2, frequency='minute',fields='price')
            prices = df[389:450]
            d[date] = prices
    return d 


In [ ]:
d = merged(results)


In [ ]:
dates = sorted(d.keys())
l = list()
for i, date in enumerate(dates):
    if i + 1 < len(dates):
        prices = d[date]
        prevclose_to_open = pd.Series(prices.ix[1] /prices.ix[0] - 1, name ="prevclose_to_open")
        open_to_high = pd.Series(prices.max() /  prices.ix[1] - 1, name = "open_to_high")
        open_to_low = pd.Series(prices.min() /  prices.ix[1] - 1, name = "open_to_low")
        open_to_close = pd.Series(prices.ix[-1] /  prices.ix[1] - 1, name = "open_to_close")
        l.append(pd.concat([prevclose_to_open, open_to_high, open_to_low, open_to_close, results.loc[date]], axis = 1))

    
    

In [ ]:
#pd.concat([l[0], results.loc[dates[0]]], axis=1)
df = pd.concat(l)

In [ ]:
df[df["prevclose_to_open"] < -0.05].plot.scatter(x = "prevclose_to_open", y = "open_to_close", c = "MarketCapMA3")

In [ ]:
df[df["prevclose_to_open"] > 0.05].describe().T

In [ ]:
df[df["prevclose_to_open"] > 0.06].plot.scatter(x = "prevclose_to_open", y = "open_to_close" , c = "VolumeMAZscore")

In [ ]:
# 前日の引け値から当日寄り付きの差

from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing 
from quantopian.pipeline.factors import SimpleMovingAverage, Returns, DailyReturns
from quantopian.pipeline.filters.morningstar import Q1500US,Q500US
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.classifiers.morningstar import Sector 
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets
from quantopian.pipeline.experimental import QTradableStocksUS

import numpy as np
import pandas as pd 

## 結果用Factors
## Open to High 
## Open to Close 

class OpenToHigh(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.high]
    window_length = 1

    def compute(self, today, assets, out, open_, high):
        out[:] = np.log(high[-1] / open_[-1])

class OpenToClose(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 1

    def compute(self, today, assets, out, open_, close):
        out[:] = np.log(close[-1] / open_[-1])
        
        
## 前日までのFactors
## 前日Closeから当日Open
## 前日のMarketCap
## 前日までのMAPctChange
## 前日のClose
## 前々日から前日のReturn

class PreviousCloseToOpen(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 2

    def compute(self, today, assets, out, open_, close):
        out[:] = np.log(open_[-1] / close[0])
        
class DayBeforeYesterdayCloseToOpen(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 3

    def compute(self, today, assets, out, open_, close):
        out[:] = np.log(open_[-1] / close[0])
        
class PreviousMarketCap(CustomFactor):
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 2

    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[0] * shares[0])

class PreviousMAPctChange(CustomFactor):
    ## 当日を外して計算するために[:-1]
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values[:-1]).pct_change().mean()#.apply(np.log)
        out[:] = mean.values
        
class PreviousMAPctChangeLog(CustomFactor):
    def compute(self, today, assets, out, values):
        df = pd.DataFrame(values[:-1])
        mean = (df.shift(1) / df).mean().apply(np.log)
        out[:] = mean.values        
        
class MAPctChange(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean()
        out[:] = mean.values

class PreviousFloatingStocks(CustomFactor):
    inputs = [USEquityPricing.volume, Fundamentals.shares_outstanding]
    window_length = 2
    def compute(self, today, assets, out, volume, shares): 
        out[:] = volume[0] / shares[0]

        
ORNINGSTAR_SECTOR_CODES = {  
        -1: 'Misc',  
        101: 'Basic Materials',  
        102: 'Consumer Cyclical',  
        103: 'Financial Services',  
        104: 'Real Estate',  
        205: 'Consumer Defensive',  
        206: 'Healthcare',  
        207: 'Utilities',  
        308: 'Communication Services',  
        309: 'Energy',  
        310: 'Industrials',  
        311: 'Technology' ,  }

    

In [ ]:
pipe = Pipeline()
pipe.add(OpenToHigh(), "OpenToHigh")
pipe.add(OpenToClose(), "OpenToClose")
# pipe.add(USEquityPricing.open.latest, "Open")
# pipe.add(USEquityPricing.close.latest, "Close")
# pipe.add(USEquityPricing.high.latest, "High")
pipe.add(PreviousCloseToOpen(),"PreviousCloseToOpen")
pipe.add(DayBeforeYesterdayCloseToOpen(), "DayBeforeYesterdayCloseToOpen")
pipe.add(PreviousMarketCap(), "PreviousMarketCap")
pipe.add(PreviousFloatingStocks(), "PreviousFloatingStocks")


term = 5
x1 = PreviousMAPctChange(inputs = [USEquityPricing.close], window_length = term + 1)
pipe.add(x1, "PreviousClosePctChangeMA%s" % term)
x2 = PreviousMAPctChangeLog(inputs = [USEquityPricing.close], window_length =  term + 1)
pipe.add(x2, "PreviousClosePctChangeMA%s_log" % term)

x3 = PreviousMAPctChange(inputs = [USEquityPricing.volume], window_length =  term + 1)
pipe.add(x3, "PreviousVolumePctChangeMA%s" % term)
x4 = PreviousMAPctChangeLog(inputs = [USEquityPricing.volume], window_length =  term + 1)
pipe.add(x4, "PreviousVolumePctChangeMA%s_log" % term)

zs = x1.zscore( mask=x1.percentile_between(1, 99),)
sector = Sector()

pipe.add(zs, 'CloseMAZscore')
pipe.add(sector, 'Sector')
# pipe.add(MAPctChange(inputs = [USEquityPricing.close], window_length = 3), "ClosePctChangeMA3")

myscreen = (x1 > 0) & (x3 > 0) 
pipe.set_screen(Q500US() & myscreen)

# assets = StaticAssets([symbols("AAPL")])
# pipe.set_screen(assets)

start = "2015-1-1"
end = "2017-1-1"
results = run_pipeline(pipe, start_date=start, end_date=end,)


In [ ]:
#[len(results.loc[d]) for d in results.index.get_level_values(0).unique()]

In [ ]:
results.columns

In [ ]:
results[(results['Sector'] == 103) & (results["PreviousClosePctChangeMA10"] > 0.005275)].plot(kind = 'scatter', x = 'PreviousCloseToOpen', 
                                       y = 'OpenToClose',xlim=[0,0.1],
                                      c = 'DayBeforeYesterdayCloseToOpen')

In [ ]:
results[results['Sector'] == 103].describe().T

In [ ]:
results[(results["DayBeforeYesterdayCloseToOpen"] > 0.05) 
       & (results['Sector'] == 103)].plot(kind='scatter', 
                                          x = "DayBeforeYesterdayCloseToOpen", 
                                          y = 'OpenToClose',
                                          c = 'PreviousVolumePctChangeMA5_log'
                                         )

In [ ]:
results[(results["DayBeforeYesterdayCloseToOpen"] > 0.05) & (results['Sector'] == 103)].describe()


In [ ]:
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.experimental import QTradableStocksUS

class ValueDaybeforeYesterday(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = values[0] 
        
class ValueYesterday(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = values[-1] 

class FloatingStocks(CustomFactor):
    inputs = [USEquityPricing.volume, Fundamentals.shares_outstanding]
    window_length = 1
    def compute(self, today, assets, out, volume, shares): 
        out[:] = volume[-1] / shares[-1]

class MAPctChange(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean()#.apply(np.log)
        out[:] = mean.values
        
class MAPctChangeLog(CustomFactor):
    def compute(self, today, assets, out, values):
        df = pd.DataFrame(values)
        mean = (df.shift(1) / df).mean().apply(np.log)
        out[:] = mean.values

        
        
pipe = Pipeline()
base_universe = QTradableStocksUS()
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(98, 100)

close_day_before_yeseterday = ValueDaybeforeYesterday(inputs = [USEquityPricing.close])
close_yeseterday = ValueYesterday(inputs = [USEquityPricing.close])
ma_volume_change = MAPctChange(inputs = [USEquityPricing.volume], window_length=3)
ma_volume_change_log = MAPctChangeLog(inputs = [USEquityPricing.volume], window_length=3)

ma_close_change = MAPctChange(inputs = [USEquityPricing.close], window_length=3)
ma_close_change_log = MAPctChangeLog(inputs = [USEquityPricing.close], window_length=3)


pipe.add(close_day_before_yeseterday, "close_day_before_yeseterday")
pipe.add(close_yeseterday, "close_yeseterday")
pipe.add(ma_volume_change, 'ma_volume_change')
pipe.add(ma_volume_change_log, 'ma_volume_change_log')
pipe.add(ma_close_change, 'ma_close_change')
pipe.add(ma_close_change_log, 'ma_close_change_log')


pipe.set_screen(base_universe & high_dollar_volume & (ma_close_change < 0))

start = "2015-1-5"
end = "2017-1-1"
results = run_pipeline(pipe, start_date=start, end_date=end,)
# aapl = get_pricing("aapl", start_date=start, end_date=end, )

In [ ]:
# このDatesは，before_trading_start 時に取ったDateと同じと考えることができる
dates = results.index.get_level_values(0).unique()
d = dict()   
for date in dates:
    print date.strftime("%Y%m%d"),
    ticker = results.loc[date].index 
    d[date] = get_pricing(ticker, start_date=date, end_date=date, frequency='minute',fields='price')[:40]
    
    

In [ ]:
len(results.loc[dates[-1]])

In [ ]:
def make_data(pipeline_result, get_pricing_result):
    open_price =  pd.Series(get_pricing_result.iloc[0], name = "open_price")
    open_to_1010 = pd.Series(get_pricing_result.iloc[-1]/get_pricing_result.iloc[0] - 1, name = "open_to_1010")
    df = pd.concat([pipeline_result,open_price,open_to_1010], axis=1)
    df["diff_close_day_before_yeseterday_to_open"] = df["open_price"] / df["close_day_before_yeseterday"] - 1
    df["diff_close_yeseterday_to_open"] = df["open_price"] / df["close_yeseterday"] - 1
    return df 

def merge_data(results, d):
    dates = results.index.get_level_values(0).unique()
    l = list()
    for date in dates:
        df = make_data(results.loc[date], d[date])
#         df = df[(df["diff_close_day_before_yeseterday_to_open"] < -0.05)
#                 |(df["diff_close_day_before_yeseterday_to_open"] > 0.05) ]
        df = df[df["diff_close_day_before_yeseterday_to_open"] > 0.05]
        #df = df.nlargest(10, "diff_close_day_before_yeseterday_to_open")
    
        l.append(df)
    return pd.concat(l)

df = merge_data(results, d)
    

In [ ]:
#df.plot.scatter(x = "volume_change_for_5days", y = 'open_to_1010')
#df[df["diff_close_day_before_yeseterday_to_open"].apply(np.sign) ==df["open_to_1010"].apply(np.sign)].plot.scatter(x = "ma_volume_change_log", y = 'open_to_1010')
df.plot.scatter(x = "diff_close_day_before_yeseterday_to_open", y = 'open_to_1010')
df